In [1]:
from movie_recommender import MovieRecommender
import timeit

In [2]:
started = timeit.default_timer()
movies_file = 'data/ml-10M100K/movies.dat'
ratings_file = 'data/ml-10M100K/ratings.dat'
movie_rec = MovieRecommender(movies_filename=movies_file, ratings_filename=ratings_file)
elapsed = timeit.default_timer() - started
print(f'Read dataframes in {elapsed}ms')
movie_rec.ratings_df.head()

testing c:\Users\john.french\movie_recs\data\pickled\movie_ratings.pkl
found pickle at c:\Users\john.french\movie_recs\data\pickled\movie_ratings.pkl
Read dataframes in 2.703391300048679ms


,user_id,rating,timestamp
movie_id,,,
122,1,5,838985046.0
185,1,5,838983525.0
231,1,5,838983392.0
292,1,5,838983421.0
316,1,5,838983392.0


In [5]:
user_id = movie_rec.get_next_avail_user_id()
# movie_rec.add_movie_rating(movie_id=1, user_id= user_id, rating= 5) # Toy Story
# movie_rec.add_movie_rating(movie_id=2, user_id= user_id, rating= 5) # Jumanji
# movie_rec.add_movie_rating(movie_id=48, user_id= user_id, rating= 5) # Pocahontas
movie_rec.add_movie_rating(movie_id=20, user_id= user_id, rating= 5) # Money Train
movie_rec.add_movie_rating(movie_id=25, user_id= user_id, rating= 5) # Leaving Las Vegas
movie_rec.add_movie_rating(movie_id=50, user_id= user_id, rating= 5) # The Usual Suspects
movie_rec.ratings_df.head()

,user_id,rating,timestamp
movie_id,,,
122,1,5,838985046.0
185,1,5,838983525.0
231,1,5,838983392.0
292,1,5,838983421.0
316,1,5,838983392.0


In [9]:
print(movie_rec.get_movie_title(20))
print(movie_rec.get_movie_title(25))
print(movie_rec.get_movie_title(50))

Money Train
Leaving Las Vegas
Usual Suspects, The


In [6]:
rec = movie_rec.get_naive_recommendation(user_id)

called get_most_similar_user
found similar user of 58
Blade Runner
Get on the Bus
Little Mermaid, The
Green Mile, The
Rebel Without a Cause
Streetcar Named Desire, A
Silence of the Lambs, The
Who's Afraid of Virginia Woolf?
Full Monty, The
Rosemary's Baby
Red Violin, The (Violon rouge, Le)
Run Lola Run (Lola rennt)
Cinema Paradiso (Nuovo cinema Paradiso)
Eyes Wide Shut
Lady Vanishes, The
Lolita
Princess Bride, The
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark)
Malcolm X
Color Purple, The
Lawrence of Arabia
Clockwork Orange, A
Amadeus
Boat, The (Das Boot)
Sting, The
Harold and Maude
8 1/2
Chinatown
M
Big
Young Frankenstein
Nashville
Great Dictator, The
Fantasia
Elephant Man, The
Life Is Beautiful (La Vita è bella)
Nights of Cabiria (Notti di Cabiria, Le)
Shakespeare in Love
Risky Business
Children of Paradise (Les enfants du paradis)
Hairspray
Vertigo
North by Northwest
Some Like It Hot
Casablanca
My Fair Lady
Wizard of Oz, The
Gone with the Wind
Sunset Blvd. (a

In [ ]:
import numpy as np


# return the difference between average ratings for two movies
def get_movie_mean_rating_error(movie1_id: int, movie2_id: int):
    movie1_ratings = np.mean(
        movie_rec.ratings_df.loc[movie_rec.ratings_df.index == movie1_id]['rating'].values)
    movie2_ratings = np.mean(
        movie_rec.ratings_df.loc[movie_rec.ratings_df.index == movie2_id]['rating'].values)
    return abs(movie1_ratings - movie2_ratings)


# get_movie_mean_rating_error(599, 700)
# movie_rec.get_movie_users(5)
print(movie_rec.get_movie_title(1))
print(movie_rec.get_movie_title(2))
print(len(movie_rec.get_movie_users(1)))
print(len(movie_rec.get_movie_users(2)))
print(len(movie_rec.get_common_users(1, 2)))

In [ ]:
def get_movie_rating_pearson_coeff(movie1_id: int, movie2_id: int)-> float:
    common_users = movie_rec.get_common_users(movie1_id=movie1_id, movie2_id=movie2_id)
    movie1_ratings = [movie_rec.get_user_movie_rating(user_id=u, movie_id=movie1_id) for u in common_users]
    movie2_ratings = [movie_rec.get_user_movie_rating(user_id=u, movie_id=movie2_id) for u in common_users]
    coeff = np.corrcoef(np.array(movie1_ratings), np.array(movie2_ratings))[0, 1]
    movie1_title = movie_rec.get_movie_title(movie1_id)
    movie2_title = movie_rec.get_movie_title(movie2_id)
    print(f'{movie1_title} and {movie2_title} have a correlation of {"{:.2f}".format(coeff)} from {len(common_users)} users who have rated both')

get_movie_rating_pearson_coeff(1, 3)